In [17]:
import requests
import pandas as pd

In [18]:
url = 'http://www.foodauthority.nsw.gov.au/penalty-notices/default.aspx?template=results'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-1] #get the last dataframe from website
df.head()

#get current column names
headers = list(df)
#print("old header names: ")
#print(headers)

#new header names
headers = ['Trade name', 'Suburb', 'Council', 'penalty_num', 'Date', 'Party served', 'Notes']

#rename headers
df.columns = headers

#change index
df.set_index('penalty_num', inplace=True)
df.head()

,Trade name,Suburb,Council,Date,Party served,Notes
penalty_num,,,,,,
3161225073,(NO TRADING NAME),MANLY,Northern Beaches,2018-03-06,MEAT ANGEL PTY LTD,NaN
3161225064,(NO TRADING NAME),MANLY,Northern Beaches,2018-03-06,MEAT ANGEL PTY LTD,NaN
3144236747,3 ADDICTIONS,NEUTRAL BAY,North Sydney,2018-04-16,"ENGINEER, BERJIZ",NaN
3144236756,3 ADDICTIONS,NEUTRAL BAY,North Sydney,2018-04-16,"ENGINEER, BERJIZ",NaN
3144236920,3 ADDICTIONS,NEUTRAL BAY,North Sydney,2018-05-14,"ENGINEER, BERJIZ",NaN


# Penalty Notes 
All the juicy info is stored in the NOTES which has a slug = penalty_num. We want to know
- the address, 
- the offence code, 
- the description (what happened?) and, 
- the fine amount

In [19]:
#dataframe to hold offences
notes=pd.DataFrame(columns=['address', 'offence_code', 'description', 'amount'])

#empty lists will eventually be columns in dataframe
address_col=[]
offence_col=[]
description_col=[]
amount_col=[]

for index, row in df.iterrows():
    penalty_num = index
    #print(penalty_num) #debugging
    
    url = 'http://www.foodauthority.nsw.gov.au/penalty-notices/default.aspx?template=detail&itemId='+str(penalty_num)
    html = requests.get(url).content
    df_list = pd.read_html(html)
    penaltydf = df_list[-1] #get the last dataframe from website
    penaltydf = penaltydf.T #transpose so that features are columns (not rows)
    penaltydf.columns=penaltydf.iloc[0] #get the header names from the first row.

    #update each series (each column-to-be)
    address_col.append(penaltydf['Address(where offence occurred)'][1])
    offence_col.append(penaltydf['Offence code'][1])
    description_col.append(penaltydf['Nature & circumstances of alleged offence'][1])
    amount_col.append(penaltydf['Amount of penalty'][1])
    
#add distane column to dataframe (by converting to a series)
df['address'] = pd.Series(address_col, df.index) 
df['offence'] = pd.Series(offence_col, df.index)
df['description'] = pd.Series(description_col, df.index)
df['amount'] = pd.Series(amount_col, df.index) 

In [21]:
df.head()

,Trade name,Suburb,Council,Date,Party served,Notes,address,offence,description,amount
penalty_num,,,,,,,,,,
3161225073,(NO TRADING NAME),MANLY,Northern Beaches,2018-03-06,MEAT ANGEL PTY LTD,NaN,249 PITTWATER ROAD MANLY 2095,11369 - Handle sell food so as to contravene t...,"Fail to maintain all fixtures, fittings and eq...",$1320
3161225064,(NO TRADING NAME),MANLY,Northern Beaches,2018-03-06,MEAT ANGEL PTY LTD,NaN,249 PITTWATER ROAD MANLY 2095,11369 - Handle sell food so as to contravene t...,Fail to maintain the food premises to the requ...,$1320
3144236747,3 ADDICTIONS,NEUTRAL BAY,North Sydney,2018-04-16,"ENGINEER, BERJIZ",NaN,"SHOP 4, 4 RANGERS ROAD NEUTRAL BAY 2089",11338 - Fail to comply with Food Standards Cod...,Fail to maintain at or near each hand washing ...,$440
3144236756,3 ADDICTIONS,NEUTRAL BAY,North Sydney,2018-04-16,"ENGINEER, BERJIZ",NaN,"SHOP 4, 4 RANGERS ROAD NEUTRAL BAY 2089",11338 - Fail to comply with Food Standards Cod...,Fail to ensure that persons undertaking or sup...,$440
3144236920,3 ADDICTIONS,NEUTRAL BAY,North Sydney,2018-05-14,"ENGINEER, BERJIZ",NaN,SHOP 4 / 4 RANGERS ROAD NEUTRAL BAY 2089,11338 - Fail to comply with Food Standards Cod...,Fail to ensure that persons undertaking or sup...,$440


## How many fines in total have been collected?
- convert amount from string to numeric
- need to remove $ sign

In [25]:
#replace the dollar sign
df['amount'] = df['amount'].replace({'\$':''}, regex = True) #regex=True because \escape is a regular expression

#convert amount to numeric
df['amount']=pd.to_numeric(df['amount'])

amount_sum = df['amount'].sum()

"${:,}".format(amount_sum)

'$1,226,940'

## What is the most common offence?
Check the mode

In [84]:
df['offence'].mode()

0    11339 - Fail to comply with Food Standards Cod...
dtype: object

In [79]:
#df['offence'].groupby('offence').count()
#df[df.Suburb=='BLACKTOWN']

# Write to csv

In [81]:
df.to_csv('food')

In [ ]:
#import csv


#find latest date


#import from website and add those AFTER that date